In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import io
import re
import regex
import csv
from datetime import datetime as dt
import urllib3
# import re2


In [2]:
url = 'https://www.opensecrets.org/races/summary.csv?cycle=2020&id=TN07'

r = requests.get(url)
data = r.content.decode('utf8')
df = pd.read_csv(io.StringIO(data))
df.head(2)

,cid,FirstLastP,Rcpts,Spent,PACs,Indivs,Cand,Other,EndCash,LgIndivs,...,Result,CRPICO,State,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote
0,N00041873,Mark Green (R),1194960.47,935486.67,171900.0,819151.42,0.0,203909.05,287888.55,819151.42,...,W,I,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,1,N
1,N00045536,Kiran Sreepada (D),206644.28,207190.98,4000.0,202644.28,0.0,0.00,0.00,179129.75,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,,0,2,N


In [3]:
dist_01 = 'https://www.opensecrets.org/races/summary.csv?cycle=2020&id=TN01'

r = requests.get(dist_01)
data = r.content.decode('utf8')
df = pd.read_csv(io.StringIO(data))
df.head(2)

,cid,FirstLastP,Rcpts,Spent,PACs,Indivs,Cand,Other,EndCash,LgIndivs,...,Result,CRPICO,State,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote
0,N00046688,Diana Harshbarger (R),2126945.60,1869099.77,222800.0,359728.50,1461293.0,83124.1,257845.83,315489.1,...,W,O,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,,0,2,N
1,N00046686,Blair Nicole Walsingham (D),140209.14,134994.55,1520.0,138689.14,0.0,0.0,5214.59,70085.2,...,L,O,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,,0,2,N


In [4]:
r

<Response [200]>

In [5]:
dist_11 = 'https://www.opensecrets.org/races/summary.csv?cycle=2020&id=TN11'

r = requests.get(dist_11)
data = r.content.decode('utf8')
df = pd.read_csv(io.StringIO(data))
df.head(2)
#try and accept running into an EmptyDataError - then fail gracefully

EmptyDataError: No columns to parse from file

In [6]:
r

<Response [200]>

In [ ]:
def pull_down_links(url):
    r = requests.get(url)
    data = r.content.decode('utf8')
    df = pd.read_csv(io.StringIO(data))
    return df
pull_down_links(url)

In [ ]:
#for link in soup.findAll('a', attrs={'href': re.compile(r'\/races\/summary\?cycle=2020&id=TN\d{2}&spec=N')}):
#    links.append(link.get('href'))

In [ ]:
#def getLinks(url):
#    html_page = requests.get(url)
#    soup = BeautifulSoup(html_page.text)
#   links = []

#    for link in soup.findAll('a', attrs={'href': re.compile(r'\/races\/summary\?cycle=2020&id=TN\d{2}&spec=N')}):
#        links.append(link.get('href'))

#    return links
#getLinks(url)

In [ ]:
#state_abbr = [
#  'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL','GA',
#  'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME','MD',
#  'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH' ,'NJ',
#  'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
#  'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY',
# ]

In [ ]:
#state_list = []
#district_list = []

#for state in state_abbr:
#     URL_State = f'https://www.opensecrets.org/races/summary.csv?cycle=2020&id={state}{}'
      
     # state_list.append(URL_State)
 

     # for URL2 in state_list:
#     links2 = getLinks(URL_State)
#     district_list.extend(links2)

In [ ]:
#ef getLinks(url):
#    r = requests.get(url)
#    data = r.content.decode('utf8')
#    df = pd.read_csv(io.StringIO(data))
#    df
#    links = []
#    for link in r.findAll('a', attrs={'href': re.compile(r'\/races\/summary\?cycle=2020&id=\w{2}\d{2}&spec=N')}):
#        links.append(link.get('href'))

#    return links
#getLinks